# Generating Fantastic Reports
### Creating a simple report in plain text

In [3]:
from datetime import datetime
TEMPLATE = '''
Movies report
-------------

Date: {date}
Movies seen in the last 30 days: {num_movies}
Total minutes: {total_minutes}
'''
data = {  # values to report dictionary
    'date' : datetime.utcnow(),
    'num_movies' : 3,
    'total_minutes' : 376,
}
report = TEMPLATE.format(**data)  #unpack dict
report

'\nMovies report\n-------------\n\nDate: 2022-08-08 03:40:46.543709\nMovies seen in the last 30 days: 3\nTotal minutes: 376\n'

Create a new file with the current date and store the report:

In [8]:
FILENAME_TMPL = "junk-{date}_report.txt" 
filename = FILENAME_TMPL.format(date = data['date'].strftime('%Y-%m-%d'))
with open(filename,'w') as file:
    file.write(report)
!cat junk-*_report.txt


Movies report
-------------

Date: 2022-08-08 02:02:36.884762
Movies seen in the last 30 days: 3
Total minutes: 376


### Using templates for reports
`jinja2==3.0.1` special HTML/Python `jinja_template.html`ß

In [6]:
from jinja2 import Template
with open('jinja_template.html') as file:
    template = Template(file.read())
context = {
    'date': datetime.now(),
    'movies': ['Casablanca', 'The Sound of Music', 'Vertigo'],
    'total_minutes': 404,
}
with open('junk-report.html', 'w') as file:
    file.write(template.render(context))


In template, `{{movies|length}}`  `length` filter applied to `movies` using pipe. E.g.
```
{% if movies|length > 5 %}
  Wow, so many movies this month!
{% else %}
  Regular number of movies
{% endif %}
```
[more jinja builtin filters](http://jinja.pocoo.org/docs/2.11/templates/#list-of-builtin-filters). Also, possible to escape HTML tags: <br> `Template('{{variable}}', autoescape=False).render({'variable': '<'}) '<'`

### Formatting text in Markdown
`mistune==0.8.4`  [Dillinger Markdown online editor](https://dillinger.io/), __[syntaX](https://daringfireball.net/projects/markdown/syntax)__, [good cheat sheet with the most frequently used elements](https://www.markdownguide.org/cheat-sheet/)

In [11]:
import mistune
with open('markdown_template.md') as file:
    template = file.read()
context = {
    'date': datetime.now(),
    'pmovies': ['Casablanca', 'The Sound of Music', 'Vertigo'],
    'total_minutes': 404,
}
context['num_movies'] = len(context['pmovies'])
context['movies'] = '\n'.join(   '* {}'.format(movie) for movie in context['pmovies']   )
md_report = template.format(**context)
report = mistune.markdown(md_report)
with open('junk-report.html', 'w') as file:
    file.write(report)

### Writing a basic Word document

In [14]:
import docx
from datetime import datetime
context = {
    'date': datetime.now(),
    'movies': ['Casablanca', 'The Sound of Music', 'Vertigo'],
    'total_minutes': 404,
}
doc = docx.Document()
doc.add_heading('Movies Report', 0)
par = doc.add_paragraph('Date: ')
par.add_run(str(context['date'])).italic = True
par = doc.add_paragraph('Movies in last 30 days: ')
par.add_run(str(context['movies'])).italic = True
for movie in context['movies']:
    doc.add_paragraph(movie, style='List Bullet')
par = doc.add_paragraph('Total minutes: ')
par.add_run(str(context['total_minutes'])).italic = True
doc.save('junk-word-report.docx')

### Styling a Word document

In [16]:
import docx
doc = docx.Document()
p = doc.add_paragraph('Different emphases: ')
p.add_run('bold').bold = True
p.add_run(', ')
p.add_run('italics').italic = True
p.add_run(' and ')
p.add_run('underline').underline = True
p.add_run('.')
doc.add_paragraph('a few', style='List Bullet')
doc.add_paragraph('bullet', style='List Bullet')
doc.add_paragraph('points', style='List Bullet')
doc.add_paragraph('Or numbered', style='List Number')
doc.add_paragraph('that will', style='List Number')
doc.add_paragraph('keep', style='List Number')
doc.add_paragraph('count', style='List Number')
doc.add_paragraph('And finish with a quote', style='Quote')

In [18]:
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor
DARK_BLUE = RGBColor.from_string('1b3866')
p = doc.add_paragraph('This par has manual styling and right alignment')
p.runs[0].font.name = 'Arial'
p.runs[0].font.size = Pt(25)
p.runs[0].font.color.rgb = DARK_BLUE
p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
doc.save('junk-word-style.docx')

If you need to generate a palette, it's a good idea to use tools such as [coolors](https://coolors.co/) to generate good combinations.

### Generating structure in Word documents

In [22]:
import docx
doc = docx.Document()
p = doc.add_paragraph('This is the start of a paragraph')
run = p.add_run()
run.add_break(docx.text.run.WD_BREAK.LINE)
p.add_run('And this is a different run')
p.add_run('. Even though on the same paragraph.')
doc.add_page_break()
doc.add_paragraph('On a new page')
sec = doc.add_section(docx.enum.section.WD_SECTION.NEW_PAGE)
sec.orientation = docx.enum.section.WD_ORIENT.LANDSCAPE
sec.page_height, sec.page_width = sec.page_width, sec.page_height
doc.add_paragraph('And this is Landscape section')
sec = doc.add_section(docx.enum.section.WD_SECTION.NEW_PAGE)
sec.orientation = docx.enum.section.WD_ORIENT.PORTRAIT
sec.page_height, sec.page_width = sec.page_width, sec.page_height
doc.add_paragraph('Revert to Portrait')
doc.save('junk-word-struct.docx')

```
from docx.shared import Inches, Cm
>>> section.page_height = Inches(10)
>>> section.page_width = Cm(20)
The page margins can also be defined in the same way:
>>> section.left_margin = Inches(1.5)
>>> section.right_margin = Cm(2.81)
>>> section.top_margin = Inches(1)
>>> section.bottom_margin = Cm(2.54)
```
Sections can also be forced to start not only on the next page, but on the next odd page, which will look better when printing on two sides:
```
>>> document.add_section( docx.enum.section.WD_SECTION.ODD_PAGE)
```

### Adding pictures to Word documents
`$ wget https://github.com/PacktPublishing/Python-Automation-Cookbook-Second-Edition/blob/master/Chapter04/images/photo-dublin-a1.jpg`


In [29]:
import docx
doc = docx.Document()
p = doc.add_paragraph('Below is a picture of Dublin')
image = doc.add_picture('images/photo-dublin-a1.jpg')
ASPECT = image.height / image.width
from docx.shared import Inches, Cm
image.width = Cm(14)
image.height = Cm(ASPECT * 14)
p = doc.paragraphs[-1]
from docx.enum.text import WD_ALIGN_PARAGRAPH
p.alignment = WD_ALIGN_PARAGRAPH.CENTER
p.add_run().add_break()
p.add_run('Pic of Dublin')
doc.add_paragraph('Keep adding text in a new paragraph')
doc.save('junk-word-pic.docx')

### Writing a simple PDF document
`fpdf==1.7.2`  [FPDF doc](http://pyfpdf.readthedocs.io/en/latest/index.html)

In [30]:
import fpdf
doc = fpdf.FPDF()
doc.set_font('Times', 'B', 14)
doc.set_text_color(19, 83, 173)
doc.add_page()
# .cell only for single line -- get_string_width to find
doc.cell(0, 5, 'PDF Test Doc') # Cell is BOX -- width=0 (page width) height=5mm
doc.ln()  # new line
doc.set_font('Times', '', 12)
doc.set_text_color(0)
doc.multi_cell(0,5, 'This is an example of a very long paragraph. ' * 10)
doc.ln()
doc.multi_cell(0, 5, 'Another long paragraph. Lorem ipsum dolor sit amet, consectetur adipis elit. ' * 20)
doc.output('junk-report.pdf')

''

### Structuring a PDF
`ch05-structuring_pdf.py`   Also, [set_link](http://pyfpdf.readthedocs.io/en/latest/reference/set_link/index.html) of [FPDF](http://pyfpdf.readthedocs.io/en/latest/index.html)

In [1]:
import fpdf
from random import randint

class StructuredPDF(fpdf.FPDF):     # no own __init__
    LINE_HEIGHT = 5

    def footer(self):  # overrides superclass
        self.set_y(-15)
        self.set_font('Times', 'I', 8)
        # doc.alias_nb_pages() in main sets {nb} . It will be substituted as the document is closed.
        page_number = 'Page {number}/{{nb}}'.format(number=self.page_no())  
        self.cell(0, self.LINE_HEIGHT, page_number, 0, 0, 'R') # border=0, ln=0, align='R'

    def chapter(self,title,paragraphs):
        self.add_page()
        link = self.title_text(title)
        page = self.page_no()
        for par in paragraphs:
            self.multi_cell(0,self.LINE_HEIGHT,paragraph)
            self.ln()
        return link, page

    def title_text(self,title)
        self.set_font('Times', 'B', 15)
        self.cell(0, self.LINE_HEIGHT, title)
        self.set_font('Times', '', 12)
        self.line(10,17,110,17)
        link = self.add_link()
        self.set_link(link)
        self.ln()
        self.ln()
        return link

    def get_full_line(self, head, tail, fill):
        pass # see .py
    def toc(self,links):
        self.add_page()
        self.TITLE_TEXT('Table of contents')
        self.set_font('Times', 'I', 12)
        for title, page, link in links:
            line = self.get_full_line(title, page, '.')
            self.cell(0, self.LINE_HEIGHT, line, link=link)
            self.ln()

LOREM_IPSUM = ('blah')

def main():
    doc = StructuredPDF()
    doc.alias_nb_pages()
    links = []
    num_chapters = randint(5,40)
    for index in range(1,num_chapters):
        chapter_title = f'Chapter {index}'
        num_par = randint(10,15)
        link, page = doc.chapter(chapter_title, [LOREM_IPSUM]*num_par)
        links.append((chapter_title,page,link))
    doc.toc(links)
    doc.output('junk-toc.pdf')

if __name__ == '__main__':
    main()

### Watermarking and encrypting a PDF
`pdf2image==1.11.0` -- before combining section. Need `brew install poppler` on Mac, [or other platform](https://github.com/Belval/pdf2image#first-you-need-pdftoppm), Before that, neeed to install homebrew: `/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"`

Finally, here's `ch05-watermarking_pdf.py`

In [32]:
import PyPDF2

def encrypt(out_pdf, password):
    in_file = open(out_pdf, 'rb')
    input_pdf = PyPDF2.PdfFileReader(in_file)
    output_pdf = PyPDF2.PdfFileWriter()
    output_pdf.appendPagesFromReader(input_pdf)
    output_pdf.encrypt(password)
    INTERMEDIATE_ENCRYPT_FILE = 'temp.pdf'
    with open(INTERMEDIATE_ENCRYPT_FILE, 'wb') as out_file:
        output_pdf.write(out_file)
    in_file.close()
    os.rename(INTERMEDIATE_ENCRYPT_FILE,out_pdf)

from PIL import Image, ImageDraw, ImageFont
def create_watermark(watermarked_by):
    WATERMARK_SIZE = (200, 200)
    mask = Image.new('L', WATERMARK_SIZE, 0) # L -- gray? 0 -- white?
    draw = ImageDraw.Draw(mask)
    font = ImageFont.load_default()
    text = 'WATERMARKED BY {}\n{}'.format(watermarked_by,datetime.now())
    draw.multiline_text((0,100), text, 55, font=font)
    
    watermark = Image.new('RGB', WATERMARK_SIZE)
    watermark.putalpha(mask)



### Aggregating PDF reports
`pdf2image==1.11.0` Combine two PDF's: `junk-report-toc.pdf` and
`python ch05-watermarking_pdf.py junk-report-toc.pdf -u automate_user -o junk-report-water.pdf`